In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [ ]:
def find_warsaw_district(input_string) -> str:
    """
    Identifies and returns a Warsaw district name from a given input string.

    The function splits the input string by commas and checks each part against a list of Warsaw district names. 
    If one of the parts matches a district name, that district name is returned. If there are no matches, 
    the function returns None.

    Parameters:
    - input_string (str): The string to be analyzed, which can contain multiple comma-separated values.

    Returns:
    - str or None: The name of a Warsaw district if found; otherwise, None.
    """
    # List of Warsaw districts
    warsaw_districts = [
        "Bemowo", "Białołęka", "Bielany", "Mokotów", "Ochota",
        "Praga-Południe", "Praga-Północ", "Rembertów", "Śródmieście",
        "Targówek", "Ursus", "Ursynów", "Wawer", "Wesoła",
        "Wilanów", "Włochy", "Wola", "Żoliborz"
    ]

    # Split the input string by commas
    parts = input_string.split(',')

    # Check each part for a match with Warsaw districts
    for part in parts:
        if part.strip() in warsaw_districts:
            return part.strip()  # Return the matching district name

    return None

In [ ]:
def extract_ad_dates(row):
    ad_info = row['announcement_date']
    
    update_date_part = [line for line in ad_info.split('\\n') if 'Aktualizacja:' in line]
    last_update = update_date_part[0].replace("('Aktualizacja: ", '').strip() if update_date_part else None
    
    added_date_part = [line for line in ad_info.split('\\n') if 'Dodano:' in line]
    ad_added = added_date_part[0].replace('Dodano: ', '').strip() if added_date_part else None
    
    return pd.Series([last_update, ad_added])

In [ ]:
def process_data(data_draw):
    
    df = data_draw.copy()
    df['district'] = df['location'].apply(lambda x: find_warsaw_district(str(x)))
    df = df[~df.district.isna()]
    
    df[['last_update', 'added_dt']] = df.apply(extract_ad_dates, axis=1)
    df.drop(['announcement_date'], axis=1, inplace=True)
    df['expired'] = 0
    df['expired_date'] = None
    to_order = ['added_dt', 'last_update', 'link', 'expired', 'expired_date']
    columns_order = to_order + [col for col in df.columns if col not in to_order]
    df = df[columns_order]
    
    return df

In [ ]:
df = pd.read_csv('data_raw\otodom_scraped_data\otodom_2025_01_05.csv')

In [ ]:
df = process_data(df)

In [2]:
import pandas as pd

In [4]:
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [8]:
main

,added_dt,last_update,link,expired,expired_date,title,rent_price,area_room_num,floor,ogrzewanie,...,approximate_coordinates,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,adv_description,district
0,22.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/piekne-mieszka...,1,2025_01_04,Piękne mieszkanie na warszawskich Skoroszach,3 000 zł\n/miesiąc\n+ Czynsz 600 zł,45m²\nWynajmę również studentom\n2 pokoje,3/6,inne,...,False,2013.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Wynajmę mieszkanie 45 metrów na warszawskim Ur...,Ursus
1,15.12.2024,21.12.2024,https://www.otodom.pl/pl/oferta/zarezerwowane-...,1,2025_01_08,[ZAREZERWOWANE] przy Metrze Daszyńskiego,3 950 zł\n/miesiąc\n+ Czynsz 690 zł,42m²\n2 pokoje,2/8,miejskie,...,False,2020.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,drzwi / okna antywłamaniowe\ndomofon / wideofon,English version below.\n\nMIESZKANIE\nNowoczes...,Wola
2,18.12.2024,18.12.2024,https://www.otodom.pl/pl/oferta/nowe-2-pokojow...,1,2025_01_02,Nowe 2-pokojowe wysoki standard| Saska Kępa|Garaż,4 700 zł\n/miesiąc\n+ Czynsz 750 zł,45m²\n2 pokoje\ntylko dla niepalących,2/4,miejskie,...,False,2024.0,tak,apartamentowiec,monitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,Do wynajęcia NOWE 2-pokojowe mieszkanie o powi...,Praga-Południe
3,14.12.2023,22.12.2024,https://www.otodom.pl/pl/oferta/3-pok-70m2-pow...,1,2025_01_09,"3 pok, 70m2, Powstańców Śl. Metro Bemowo, Garaż",4 200 zł\n/miesiąc\n+ Czynsz 911 zł,70m²\n3 pokoje\ntylko dla niepalących,3/7,miejskie,...,False,2008.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,BEZPOŚREDNIO OD WŁAŚCICIELA - dostępne - Najem...,Bemowo
4,4.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/ostoja-wilanow...,0,NaN,"Ostoja Wilanów, 3 pokoje, 2 miejsca post. w ce...",7 300 zł\n/miesiąc\n+ Czynsz 1 040 zł,73m²\n3 pokoje,2/4,miejskie,...,False,2016.0,tak,apartamentowiec,monitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Komfortowy apartament wykończony w wysokim sta...,Wilanów
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18281,NaN,NaN,https://www.otodom.pl/pl/oferta/ok-placu-zbawi...,0,NaN,"Ok. Placu Zbawiciela, 4 pokoje, kuchnia, jadal...",4 300 zł\n/miesiąc,83m²\nWynajmę również studentom\n4 pokoje,1/4,miejskie,...,False,1936.0,tak,kamienica,NaN,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,domofon / wideofon,Do wynajęcia czteropokojowe mieszkanie w kamie...,Śródmieście
18282,NaN,NaN,https://www.otodom.pl/pl/oferta/mlociny-nowocz...,0,NaN,Młociny / Nowoczesne / Parking Podziemny,3 700 zł\n/miesiąc,40m²\n2 pokoje,3/6,miejskie,...,False,NaN,tak,apartamentowiec,monitoring / ochrona,meble\npiekarnik\nlodówka\npralka\nzmywarka,internet,domofon / wideofon,Oferuję na wynajem przestronne mieszkanie o po...,Bielany
18283,NaN,NaN,https://www.otodom.pl/hpr/pl/oferta/mokotow-2-...,0,NaN,Mokotów/2 pokoje/garaż/,3 500 zł\n/miesiąc,47.72m²\n2 pokoje\ntylko dla niepalących,4/6,miejskie,...,False,NaN,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,domofon / wideofon,Oferta tylko w naszym biurze - mieszkanie dost...,Mokotów
18284,NaN,NaN,https://www.otodom.pl/pl/oferta/2-pokoje-ul-ka...,0,NaN,"2 pokoje, ul. Kartograficzna 86A, Białołęka",2 600 zł\n/miesiąc,44m²\n2 pokoje,parter/3,NaN,...,True,2015.0,nie,apartamentowiec,monitoring / ochrona,meble\npiekarnik\nlodówka\npralka\nzmywarka,NaN,domofon / wideofon,"Oferuję do wynajęcia mieszkanie 2-pokojowe, zl...",Białołęka


In [6]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_05',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_14',
 '2025_01_15',
 '2025_01_16',
 '2025_01_17',
 '2025_01_18',
 '2025_01_19',
 '2025_01_20',
 '2025_01_21',
 '2025_01_22',
 '2025_01_23',
 '2025_01_24',
 '2025_01_25',
 '2025_01_26',
 '2025_01_27',
 '2025_01_28',
 '2025_01_29',
 '2025_01_30',
 '2025_01_31',
 '2025_02_01',
 '2025_02_02',
 '2025_02_03',
 '2025_02_04',
 '2025_02_06',
 '2025_02_08',
 '2025_02_09',
 '2025_02_12',
 '2025_02_13',
 '2025_02_15',
 '2025_02_16',
 '2025_02_19',
 '2025_02_20',
 '2025_02_21',
 '2025_02_22',
 '2025_02_23',
 '2025_02_24',
 '2025_02_25',
 '2025_02_26',
 '2025_02_27',
 '2025_02_28']

In [8]:
main['expired_date'] = main['expired_date'].replace('2025_02_27', '2025_02_26')

In [358]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_05',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_14',
 '2025_01_15',
 '2025_01_16',
 '2025_01_17',
 '2025_01_18',
 '2025_01_19',
 '2025_01_20',
 '2025_01_21',
 '2025_01_22',
 '2025_01_23',
 '2025_01_24',
 '2025_01_25',
 '2025_01_26',
 '2025_01_27',
 '2025_01_28',
 '2025_01_29',
 '2025_01_30',
 '2025_01_31',
 '2025_02_01',
 '2025_02_02',
 '2025_02_03',
 '2025_02_04',
 '2025_02_06',
 '2025_02_08',
 '2025_02_09',
 '2025_02_10',
 '2025_02_11',
 '2025_02_12',
 '2025_02_13',
 '2025_02_15',
 '2025_02_16',
 '2025_02_17',
 '2025_02_19',
 '2025_02_20',
 '2025_02_21',
 '2025_02_22',
 '2025_02_23',
 '2025_02_24',
 '2025_02_25',
 '2025_02_26',
 '2025_02_27',
 '2025_02_28']

In [ ]:
main = pd.concat([main, df], ignore_index=True)

In [356]:
path = f'data_processed/main.csv'
main.to_csv(path,
          encoding='utf-8',
          index=False)

### ====

check for expired date

In [10]:
import pandas as pd
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [354]:
date = '2025_02_28'
df = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date}.csv')

df = df[df.expired.eq(True)]
check = pd.merge(df, main[['link', 'expired_date', 'district']], how='left', on='link')

check['check'] = [True if a==b else False for a,b in zip(check['expired_date_x'], check['expired_date_y'])]
check[check.check.eq(False)]

,link,expired,expired_date_x,expired_date_y,district,check


In [202]:
main.update(main[['link']].merge(df, on='link', how='left'), overwrite=True)

In [ ]:
date2 = '2025_01_25'
df2 = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date2}.csv')
df2 = df2[df2.expired.eq(True)]

In [ ]:
df = check = pd.merge(df1, df2, how='outer', suffixes=('_24', '_25'), on='link')

In [210]:
date1 = '2025_02_24'
df1 = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date1}.csv')
df1 = df1[df1.expired.eq(True)]

date = '2025_02_10'
path = f'data_raw/search_for_inactive/search_for_inactive_{date}.csv'
df = pd.read_csv(path)

In [212]:
df = df[~df.link.isin(df1.link)]
df[df.expired.eq(True)]

,link,expired,expired_date
61,https://www.otodom.pl/pl/oferta/piekne-dwupozi...,1,2025_02_10
87,https://www.otodom.pl/pl/oferta/zlota-44-wysok...,1,2025_02_10
130,https://www.otodom.pl/pl/oferta/3-rooms-after-...,1,2025_02_10
152,https://www.otodom.pl/pl/oferta/apartament-z-b...,1,2025_02_10
171,https://www.otodom.pl/pl/oferta/luxury-apartme...,1,2025_02_10
...,...,...,...
5976,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,1,2025_02_10
6024,https://www.otodom.pl/pl/oferta/nowe-30m2-z-ba...,1,2025_02_10
6034,https://www.otodom.pl/pl/oferta/2-pok-lucka-ci...,1,2025_02_10
6106,https://www.otodom.pl/pl/oferta/mieszkanie-prz...,1,2025_02_10


In [214]:
df.to_csv(path,
          encoding='utf-8',
          index=False)

### ====

fill date added

In [ ]:
new_records = process_data(new_records)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
main = pd.concat([main, new_records], ignore_index=True)


In [364]:
import pandas as pd
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [366]:
main

,added_dt,last_update,link,expired,expired_date,title,rent_price,area_room_num,floor,ogrzewanie,...,approximate_coordinates,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,adv_description,district
0,22.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/piekne-mieszka...,1,2025_01_04,Piękne mieszkanie na warszawskich Skoroszach,3 000 zł\n/miesiąc\n+ Czynsz 600 zł,45m²\nWynajmę również studentom\n2 pokoje,3/6,inne,...,False,2013.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Wynajmę mieszkanie 45 metrów na warszawskim Ur...,Ursus
1,15.12.2024,21.12.2024,https://www.otodom.pl/pl/oferta/zarezerwowane-...,1,2025_01_08,[ZAREZERWOWANE] przy Metrze Daszyńskiego,3 950 zł\n/miesiąc\n+ Czynsz 690 zł,42m²\n2 pokoje,2/8,miejskie,...,False,2020.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,drzwi / okna antywłamaniowe\ndomofon / wideofon,English version below.\n\nMIESZKANIE\nNowoczes...,Wola
2,18.12.2024,18.12.2024,https://www.otodom.pl/pl/oferta/nowe-2-pokojow...,1,2025_01_02,Nowe 2-pokojowe wysoki standard| Saska Kępa|Garaż,4 700 zł\n/miesiąc\n+ Czynsz 750 zł,45m²\n2 pokoje\ntylko dla niepalących,2/4,miejskie,...,False,2024.0,tak,apartamentowiec,monitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,Do wynajęcia NOWE 2-pokojowe mieszkanie o powi...,Praga-Południe
3,14.12.2023,22.12.2024,https://www.otodom.pl/pl/oferta/3-pok-70m2-pow...,1,2025_01_09,"3 pok, 70m2, Powstańców Śl. Metro Bemowo, Garaż",4 200 zł\n/miesiąc\n+ Czynsz 911 zł,70m²\n3 pokoje\ntylko dla niepalących,3/7,miejskie,...,False,2008.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,domofon / wideofon,BEZPOŚREDNIO OD WŁAŚCICIELA - dostępne - Najem...,Bemowo
4,4.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/ostoja-wilanow...,0,NaN,"Ostoja Wilanów, 3 pokoje, 2 miejsca post. w ce...",7 300 zł\n/miesiąc\n+ Czynsz 1 040 zł,73m²\n3 pokoje,2/4,miejskie,...,False,2016.0,tak,apartamentowiec,monitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Komfortowy apartament wykończony w wysokim sta...,Wilanów
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18281,NaN,NaN,https://www.otodom.pl/pl/oferta/ok-placu-zbawi...,0,NaN,"Ok. Placu Zbawiciela, 4 pokoje, kuchnia, jadal...",4 300 zł\n/miesiąc,83m²\nWynajmę również studentom\n4 pokoje,1/4,miejskie,...,False,1936.0,tak,kamienica,NaN,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,domofon / wideofon,Do wynajęcia czteropokojowe mieszkanie w kamie...,Śródmieście
18282,NaN,NaN,https://www.otodom.pl/pl/oferta/mlociny-nowocz...,0,NaN,Młociny / Nowoczesne / Parking Podziemny,3 700 zł\n/miesiąc,40m²\n2 pokoje,3/6,miejskie,...,False,NaN,tak,apartamentowiec,monitoring / ochrona,meble\npiekarnik\nlodówka\npralka\nzmywarka,internet,domofon / wideofon,Oferuję na wynajem przestronne mieszkanie o po...,Bielany
18283,NaN,NaN,https://www.otodom.pl/hpr/pl/oferta/mokotow-2-...,0,NaN,Mokotów/2 pokoje/garaż/,3 500 zł\n/miesiąc,47.72m²\n2 pokoje\ntylko dla niepalących,4/6,miejskie,...,False,NaN,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,internet,domofon / wideofon,Oferta tylko w naszym biurze - mieszkanie dost...,Mokotów
18284,NaN,NaN,https://www.otodom.pl/pl/oferta/2-pokoje-ul-ka...,0,NaN,"2 pokoje, ul. Kartograficzna 86A, Białołęka",2 600 zł\n/miesiąc,44m²\n2 pokoje,parter/3,NaN,...,True,2015.0,nie,apartamentowiec,monitoring / ochrona,meble\npiekarnik\nlodówka\npralka\nzmywarka,NaN,domofon / wideofon,"Oferuję do wynajęcia mieszkanie 2-pokojowe, zl...",Białołęka


In [372]:
from toolkit import *

In [460]:
date = '2025_02_19'
df = pd.read_csv(f'data_raw/otodom_scraped_data/otodom_{date}.csv')

new_records = process_data(df)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
# main = pd.concat([main, new_records], ignore_index=True)

main.loc[main.link.isin(df.link), 'added_dt'] = date
main[main.link.isin(df.link)].added_dt.unique()

array(['2025_02_19'], dtype=object)

In [446]:
main.loc[15199:15366, 'added_dt'] = '2025_02_14'

In [448]:
main.loc[15891:16145, 'added_dt'] = '2025_02_18'

In [464]:
path = f'data_processed/main.csv'
main.to_csv(path,
          encoding='utf-8',
          index=False)

In [466]:
main[main.added_dt.isna()]

,added_dt,last_update,link,expired,expired_date,title,rent_price,area_room_num,floor,ogrzewanie,...,approximate_coordinates,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,adv_description,district
